In [1]:
import sys
from sys import stdout as cout
from scipy.stats import norm
from math import *
import numpy as np
import pandas as pd
from timer import Timer 
from torch import seed
from stats1 import stats
from config import get_input_parms, loadConfig
from finder import *
from HW import * 
from P_0T import discount_curve as zc
import matplotlib as plt
from ir import *

In [2]:
curve = [ ( 0.5147,  -0.1431/100.)
        , ( 1.0103,  -0.1452/100.)
        , ( 2.0096,  -0.1362/100.)
        , ( 3.0090,  -0.0973/100.)
        , ( 4.0138,  -0.0127/100.)
        , ( 5.0104,   0.0727/100.)
        , ( 6.0097,   0.1873/100.)
        , ( 7.0090,   0.3105/100.)
        , ( 8.0111,   0.4374/100.)
        , ( 9.0159,   0.5591/100.)
        , (10.0125,   0.6747/100.)
        , (12.0112,   0.8599/100.)
        , (15.0147,   1.0640/100.)
        , (20.0168,   1.2259/100.)
        , (25.0108,   1.2648/100.)
        , (30.0102,   1.2637/100.)
        , (40.0118,   1.2239/100.)
        , (50.0106,   1.1329/100.)
        ]

In [3]:
data=pd.DataFrame(curve)
data

,0,1
0,0.5147,-0.001431
1,1.0103,-0.001452
2,2.0096,-0.001362
3,3.0090,-0.000973
4,4.0138,-0.000127
5,5.0104,0.000727
6,6.0097,0.001873
7,7.0090,0.003105
8,8.0111,0.004374
9,9.0159,0.005591


In [4]:
Nt = 22   # 
nr = 12
p = 20
Yrs  = 1
#strike = keywords["strike"]
seed   = 1
gamma =0.5
sigma = 0.05
curve = curve
tn = 2       #maturity option
N  = 500000
print("Number of simulations:", N)
#Dt = Yrs/Nt
Dt=1

Number of simulations: 500000


In [5]:
Obj = np.random.RandomState()
Obj.seed(seed)

In [6]:
##Costruisco l'oggetto della discount curve

In [7]:
dc  = zc(curve = curve) 
P_0T  = np.ndarray(shape=(Nt+1), dtype=np.double) 

In [8]:
for n in range(Nt+1):
    P_0T[n] = dc.P_0t( n*Dt)         #P(0,t)=pn exp(fn(t))
nm= pd.DataFrame(P_0T)
nm

,0
0,1.000000
1,1.001453
2,1.002729
3,1.002930
4,1.000543
5,0.996407
6,0.988880
7,0.978565
8,0.965708
9,0.951073


## Swap Rate

In [9]:
t=0
def a(tn,p):
    a=0
    for i in range(1,p+1):
       a+=dc.P_0t(tn+i)
    return a
k= (dc.P_0t(tn)-dc.P_0t(tn+p)) / a(tn,p) 
print("k value for IRS fair: ", k)

k value for IRS fair:  0.013577079671993385


In [10]:
ks=[k, k+0.5/100, k-0.5/100, k+1/100, k-1/100, k+5/100, k-5/100]

## Jamshidian 

In [11]:
hw = HW(gamma=gamma, sigma=sigma)

In [12]:
print("%6.7s  %12.9s  %10.7s \n"%("k","SW price","r*"))
for el in ks:
    SW_price, r_star= hw.Swaption(tn, p, el, dc)
    print("%6.7f  %12.7f  %12.7f \n"%(el,SW_price,r_star))

     k      SW price          r* 

0.0135771     0.0363458    -0.0041311 

0.0185771     0.0979161     0.0396215 

0.0085771     0.0070509    -0.0513620 

0.0235771     0.1796133     0.0803751 

0.0035771     0.0005082    -0.1026742 

0.0635771     0.8914403     0.3334317 

-0.0364229     0.0000000    -0.8563019 



## MonteCarlo

Bank Account numerair

In [13]:
T0 = Timer();                       
T0.start()
X,Ix = hw_evol( Obj, hw, Dt, p, N)
i_phi = hw.IntPhi(dc, Dt, Nt)   
t_end = T0.stop()
print("@ elapsed %8.4f sec.\n" %(t_end))

@ elapsed   1.2483 sec.



In [14]:
Xo = X[tn]    
f = Ix[tn]
df2 = exp(i_phi[tn])*np.exp(-f)

In [15]:
print("%6.7s  %12.7s +/- %2.7s   %16.7s \n"%("k","BAN","err","N"))
for el in ks: 
    func=0
    for i in range(1,p):
        P_t= hw.BondPrice( tn, tn+i, dc, Xo)    
        func += el * P_t 
    
    func += (1+el)* hw.BondPrice(tn, tn+p, dc, Xo)
    put   = np.maximum(func - 1.0, 0.0)  # array di valori di payoff
    put   = df2*put 
    #m, S2=stats(put)
    ePut  = np.add.reduce( put * put, 0)/N
    put   = np.add.reduce( put , 0)/N
    ePut  = 3*np.sqrt( np.maximum( ePut - put*put , 0.0)/N )       
    print("%6.7f  %10.7f +/- %8.7f   %8.7s \n"%(el,put,ePut,N))

     k           BAN +/- err                  N 

0.0135771   0.0363248 +/- 0.0002512     500000 

0.0185771   0.0978539 +/- 0.0003896     500000 

0.0085771   0.0070607 +/- 0.0001070     500000 

0.0235771   0.1794917 +/- 0.0004782     500000 

0.0035771   0.0005179 +/- 0.0000263     500000 

0.0635771   0.8912173 +/- 0.0008881     500000 

-0.0364229   0.0000000 +/- 0.0000000     500000 



°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

P(t,tn) numerair

In [16]:
To = Timer()                    ## P(t,T) NUMERAIR. SECONDA PARTE ESERCIZIO 4 
To.start()
X = hw_evol_P_0T( Obj, hw, Dt, Nt, tn, N)
t_end = To.stop()
print("@ elapsed %8.4f sec.\n" %(t_end))
Xo  = X[tn]

@ elapsed   0.5651 sec.



In [17]:
print("%6.7s  %12.7s +/- %2.7s   %16.7s\n"%("k","P(t,tn)","err","N"))
P_0t= dc.P_0t(tn)
T_=[tn, tn+p]
for el in ks:
    pay=0
    for i in range(1,p):
        P_t= hw.BondPrice( tn, tn+i, dc, Xo)
        pay += el*P_t 
        
    pay += (1+el)*hw.BondPrice(tn, tn+p, dc, Xo) 

    opt1 = dc.P_0t(T_[0])*np.maximum( (pay - 1.0) , 0.0)   
    put1,s1 = stats(opt1)
    err1 = 3*sqrt( s1/N )
    print("%6.7f  %10.7f  +/- %8.7f   %8.7s \n"%(el,put1,err1,N))

     k       P(t,tn) +/- err                  N

0.0135771   0.0364791  +/- 0.0010076     500000 

0.0185771   0.0980873  +/- 0.0012480     500000 

0.0085771   0.0071430  +/- 0.0006572     500000 

0.0235771   0.1798165  +/- 0.0013685     500000 

0.0035771   0.0005229  +/- 0.0003214     500000 

0.0635771   0.8917746  +/- 0.0017358     500000 

-0.0364229   0.0000000  +/- 0.0000000     500000 



°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


P(t,tn+p) numerair

In [18]:
To = Timer()                   
To.start()
X = hw_evol_P_0T( Obj, hw, Dt, Nt, tn+p, N)
t_end = To.stop()
print("@ elapsed %8.4f sec.\n" %(t_end))
Xo  = X[tn]

@ elapsed   0.3674 sec.



In [19]:
print("%6.7s  %14.9s +/- %6.7s   %15.7s\n"%("k","P(t,tn+p)","err","N"))
P_tT = hw.BondPrice( tn, tn+p, dc, Xo)
for el in ks:
    pay=0
    for i in range(1,p):
        P_t= hw.BondPrice( tn, tn+i, dc, Xo)
        pay += el*P_t 
        
    pay += (1+el)*hw.BondPrice(tn, tn+p, dc, Xo) 
    a=np.maximum( (pay - 1.0) , 0.0)
    opt2 = dc.P_0t(tn+p)* a / P_tT
    put2, sq= stats(opt2)
    err2 = 3*sqrt( sq/N )
    
    print(" %6.7f  %10.7f +/- %8.7f  %8.7s \n"%(el,put2,err2,N))

     k       P(t,tn+p) +/-    err                 N

 0.0135771   0.0365265 +/- 0.0009542    500000 

 0.0185771   0.0981585 +/- 0.0011683    500000 

 0.0085771   0.0071476 +/- 0.0006211    500000 

 0.0235771   0.1798782 +/- 0.0012496    500000 

 0.0035771   0.0005219 +/- 0.0002993    500000 

 0.0635771   0.8916945 +/- 0.0012321    500000 

 -0.0364229   0.0000000 +/- 0.0000000    500000 

